# **SQL Practice**

# Objective
For this mini-project, students are presented with a series of questions and required to write SQL queries to answer the questions. 

The database used for this exercise is a fake database for a Country Club. It contains three tables:
- **Members,** containing member rows related to columns for the member id, surname (last name), first name, address, zipcode, telephone, recommended by, and join date 
- **Bookings,** containing booking rows related to columns for the booking id, facility id, member id, start time, and slots reserved
- **Facilities,** containing facility rows related to columns for the facility id, name, member's cost, guest cost, initial outlay expense, and monthly maintenance expense 

# Presentation
The SQL queries used to query the Springboard mySQL database are provided in the .sql document **jbush_sql_project.sql.** This notebook is written to pursue the, 'Excellence,' designation for the assignment:
- Download the raw files;
- Set up a local database;
- Load in the data;
- Use Jupyter notebook to set up a connection and query the data;

# Overview
This notebook will be broken up into THREE sections:
1. Loading the Data into a Local Database
2. Running the Queries
3. Suggestions for Improvement

## Loading the Data into a Local Database
The demo database has been downloaded as the sql file `country_club.sql`. Also, the individual tables have been downloaded as unique .csv files. These files are located in this directory under `./data/` folder.

Below, we'll demonstrate one way to create the database locally: bring in the .csv files as tables for the database. After that, we'll go over how to create the database locally using the .sql file - and use that connection object to query the database with our answers.

To complete this project, we'll need to import both the `sqlite3` and `pandas` libraries.

In [1]:
import sqlite3
import pandas as pd

### Working with tables as .csv
First, let's cover bringing in tables to the database as .csv files. While we won't use this method for this project, it's useful to review. To bring in tables to the database, we'll do the following:
1. Load the .csv into a pandas df
2. Create a `Connection` object that represents the database
3. Pass the df to the database using the pandas `.to_sql()` method
4. Close the `Connection`

With the `pandas` library imported, we'll bring in the first table *Members.csv* by passing it's filename to `pd.read_csv()`

In [2]:
# load data into a pandas df
df = pd.read_csv('./data/Members.csv', index_col=False)

> To avoid unnecessary indexes duplicating, we'll set the `index_col=` argument to **False**. We'll also set the `.to_sql()` argument `index=` later to False when placing the DataFrame in the database.

Next, let's create a `Connection` object, **con**, that represents the local database. Let's name this database **db_from_csv.** So that we create the database in our data folder, we'll assign the string *./data/db_from_csv.db* to a variable **db**, and pass that variable to the `Connection` object.

In [3]:
# database name string
db = './data/db_from_csv.db'
# create Connection object
con = sqlite3.connect(db)

With the connection established, we'll call the pandas method `.to_sql()` on the dataframe we created earlier from the csv, **df**. With `df.to_sql()`, we'll pass 3 arguments:
- The table name as a string, "Members"
- The connection object (in this case, connection object variable), **con**
- (Optional) Set the `index=` to False, so the DataFrame index is written as a column

After that, we'll also close the `Connection` by calling `.close()` on **con**

In [4]:
# create the 'Members' Table
df.to_sql("Members", con=con, index=False)
# close the connection
con.close()

Let's check the first 5 entries of the table by passing an SQL query. Don't worry about the process here, we'll touch on querying the database later.

In [5]:
# send query with content manager
with sqlite3.connect(db) as con:
    q =""" SELECT * 
           FROM Members
           LIMIT 5
       """
    r = pd.read_sql_query(q, con)
# call the return variable
r

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,NaN,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,NaN,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,NaN,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,NaN,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1.0,2012-07-03 10:25:05


*Awesome!* To input the other two tables, we'll use the function below.

In [6]:
def csv_to_table(db, fn, tn):
    """Reads .csv into pandas df, then with content manager
    establishes the connection object and passes the df
    to the database with pandas .to_sql()"""
    # fn = filename string
    # tn = table name string
    # db = database string
    df = pd.read_csv(fn, index_col=False)
    with sqlite3.connect(db) as con:
        df.to_sql(tn, con=con, index=False)

In [7]:
# Create the Facilities table from a csv
csv_to_table(db, './data/Facilities.csv', "Facilities")
# Create the Bookings table from a csv
csv_to_table(db, './data/Bookings.csv', "Bookings")

To check the tables, we'll query both tables for 2 lines and print the returns:

In [8]:
# send query with content manager
with sqlite3.connect(db) as con:
    q =""" SELECT * 
           FROM Facilities
           LIMIT 2
       """
    r = pd.read_sql_query(q, con)
# print the return variable
print(r)
# send query with content manager
with sqlite3.connect(db) as con:
    q =""" SELECT * 
           FROM Bookings
           LIMIT 2
       """
    r = pd.read_sql_query(q, con)
# print the return variable
print(r)

   facid            name  membercost  guestcost  initialoutlay  \
0      0  Tennis Court 1         5.0       25.0          10000   
1      1  Tennis Court 2         5.0       25.0           8000   

   monthlymaintenance  
0                 200  
1                 200  
   bookid  facid  memid            starttime  slots
0       0      3      1  2012-07-03 11:00:00      2
1       1      4      1  2012-07-03 08:00:00      2


*Fantastic!* Looks like we've successfully setup our database locally using .csv to create the tables. But, what if we have a .sql file? Let's look at that next!

### Working with an .sql file

An .sql file typically is a file with SQL commands for creating, editing, or deleting fields in a database. Let's print out a few of the lines below and see what we're talking about:

In [9]:
# open the file as readable text, read the lines into a list *lines*
with open('./data/country_club.sql','r') as f:
    lines = f.readlines()
# print out lines 21 to 28
for l in lines[20:28]:
    print(l)

CREATE TABLE IF NOT EXISTS "Bookings" (

  "bookid" int(4) NOT NULL DEFAULT '0',

  "facid" int(1) DEFAULT NULL,

  "memid" int(2) DEFAULT NULL,

  "starttime" varchar(19) DEFAULT NULL,

  "slots" int(2) DEFAULT NULL,

  PRIMARY KEY ("bookid")

);



Above, we see a chunk of text that contains the SQL commands used to create the table structure for the *Bookings* table. 

Below, we'll print out one more excerpt that prints out a few lines of the SQL command where values are being inserted into the *Bookings* table:

In [10]:
# open the file as readable text, read the lines into a list *lines*
with open('./data/country_club.sql','r') as f:
    lines = f.readlines()
# print out lines 34 to 39
for l in lines[33:39]:
    print(l)

INSERT INTO `Bookings` (`bookid`, `facid`, `memid`, `starttime`, `slots`) VALUES

(0, 3, 1, '2012-07-03 11:00:00', 2),

(1, 4, 1, '2012-07-03 08:00:00', 2),

(2, 6, 0, '2012-07-03 18:00:00', 2),

(3, 7, 1, '2012-07-03 19:00:00', 2),

(4, 8, 1, '2012-07-03 10:00:00', 1),



### Reading an .sql file into a local database

Since the .sql file is a collection of SQL commands that create the structure and values for the database, we'll go about it a tad differently:
1. Create a `file object` with the `open()` function, and pass the filename and the 'r' to designate the object as read only
2. Use the `.read()` method on the file object to read in the entire file
3. Create the database `Connection` object (Which also creates the database if it didn't exist)
4. Create a `Cursor` object, and call the `.executescript()` method to execute all the commands in sequence

In [11]:
# Create file object, read in text
with open('./data/country_club.sql','r') as f:
    sql = f.read()
# Create connection, cursor, database
with sqlite3.connect('./data/db_from_sql.db') as con:
    c = con.cursor()
    c.executescript(sql)

> As seen above, a connection manager can be established by using the syntax `with [function] as [variable]:` 
>
>
> With this approach, the file is properly closed after its suite finishes, and is considered good practice.

Now that we've created the local database `/data/db_from_sql.db` using the .sql file, let's send a query to check it:

In [12]:
# assign the sql database string to the 'db' variable
db = './data/db_from_sql.db'
# send query with content manager
with sqlite3.connect(db) as con:
    q =""" 
    SELECT * 
    FROM Members
    LIMIT 5
    """
    r = pd.read_sql_query(q, con)
# call the return variable
r

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


*Epic!* Now that we've confirmed a few entries of the database, let's move on to querying the database!

## Querying the Database
Before we start sending off queries, let's touch on the methods used and a few differences between SQL and SQLite syntax.

First off, remember that `Cursor` object from earlier? Well, we can pass it an SQL query using the `.execute()` method to return an iterable object. By calling the method `.fetchall()` on the `Cursor` object **c**, we can save the return to a variable for later.

Below, we'll save the query return to the variable **fetched**, and after closing the connection, print out each entry using a for loop to iterate over the variable.

In [13]:
# establish connection to database
con = sqlite3.connect('./data/db_from_sql.db')
# create cursor object
c = con.cursor()
# pass query
c.execute(""" 
SELECT * 
FROM Members
LIMIT 5
"""
          )
# save the entire query return
fetched = c.fetchall()
# close connection
con.close()
# print each row
for row in fetched:
    print(row)

(0, 'GUEST', 'GUEST', 'GUEST', 0, '(000) 000-0000', '', '2012-07-01 00:00:00')
(1, 'Smith', 'Darren', '8 Bloomsbury Close, Boston', 4321, '555-555-5555', '', '2012-07-02 12:02:05')
(2, 'Smith', 'Tracy', '8 Bloomsbury Close, New York', 4321, '555-555-5555', '', '2012-07-02 12:08:23')
(3, 'Rownam', 'Tim', '23 Highway Way, Boston', 23423, '(844) 693-0723', '', '2012-07-03 09:32:15')
(4, 'Joplette', 'Janice', '20 Crossing Road, New York', 234, '(833) 942-4710', '1', '2012-07-03 10:25:05')


> The object can also be iterated over one at a time with the method `.fetchone()`

However, pandas has a method that makes this process a bit cleaner - and automatically manages the connection: `.read_sql_query()`

To use this method, we simply pass the query and the connection object to `.read_sql_query()`, and pandas returns the query return as a DataFrame:

In [14]:
# establish connection to database
con = sqlite3.connect('./data/db_from_sql.db')
# save query to variable 'q'
q =""" 
SELECT * 
FROM Members
LIMIT 5
"""
# assign the return to variable 'df'
df = pd.read_sql_query(q, con)
df

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


*Astounding!* From now on, we'll use the **q** variable to pass each answer query to `.read_sql_query()` and call the DataFrame return to see the result (Like above). Before we get into answering questions, it's worth noting that the syntax in the following queries was adjusted slightly from SQL to SQLite. The two primary differences below occur in questions 7-10, and deal with:
- Substituting SQLite's "||" for SQL's "CONCAT()"
- Adjusting SQL's "SUBSTRING()" to SQLite's "SUBSTR()"

Also, the queries **are being passed inside triple quotes:** """ """. This allows string literals to span multiple lines of text - letting us outline the SQL queries to better see the structure of each call.

Now that we've got the details out of the way, Let's answer some questions!

## **Q1:** Some of the facilities charge a fee to members, but some do not.
## Please list the names of the facilities that do.

>Let's return the facility names that have a member cost > 0 (Or not zero; != 0).

In [15]:
# query
q ="""
SELECT name 
FROM Facilities 
WHERE membercost > 0
"""
# DataFrame of return
df = pd.read_sql_query(q, con)
df

,name
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


## **Q2:** How many facilities do not charge a fee to members?
> Here we'll COUNT the number of returns where the member cost = 0.

In [16]:
# query
q ="""
SELECT COUNT(*) AS no_fee_count
FROM Facilities 
WHERE membercost = 0
"""
# DataFrame of return
df = pd.read_sql_query(q, con)
df

,no_fee_count
0,4


## **Q3:** How can you produce a list of facilities that charge a fee to members,
## where the fee is less than 20% of the facility's monthly maintenance cost?
## Return the facid, facility name, member cost, and monthly maintenance of the
## facilities in question.
> We want to return: *facid, facility name, member cost, monthly maintenance*
>
> from *facilities* 
>
> that charge a member fee < 20% of monthly maintenance cost (Or < .2 * monthly maintenance cost).

In [17]:
# query
q ="""
SELECT facid, 
	   name, 
	   membercost, 
	   monthlymaintenance
FROM Facilities
WHERE membercost > 0
AND membercost < (.2 * monthlymaintenance)
"""
# DataFrame of return
df = pd.read_sql_query(q, con)
df

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


## **Q4:** How can you retrieve the details of facilities with ID 1 and 5?
## Write the query without using the OR operator.
> Instead of using the OR operator here, we'll use the IN operator and pass just the IDs we're interested in.

In [18]:
# query
q ="""
SELECT *
FROM Facilities
WHERE facid IN (1, 5)
"""
# DataFrame of return
df = pd.read_sql_query(q, con)
df

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


### **Q5:** How can you produce a list of facilities, with each labelled as
### 'cheap' or 'expensive', depending on if their monthly maintenance cost is
### more than $100? Return the name and monthly maintenance of the facilities
### in question.
> Using CASE WHEN allows us to use 'if-then' conditional logic.
> 
> Since there are only two 'cases' (i.e. More than 100, less or = 100) we can establish the first case conditional, then create an ELSE condition to catch the second case.

In [19]:
# query
q ="""
SELECT name, 
	   monthlymaintenance,
	   CASE WHEN monthlymaintenance <= 100 THEN 'cheap'
			ELSE 'expensive' END AS cheap_or_expensive
FROM Facilities
"""
# DataFrame of return
df = pd.read_sql_query(q, con)
df

,name,monthlymaintenance,cheap_or_expensive
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


## **Q6:** You'd like to get the first and last name of the last member(s)
## who signed up. Do not use the LIMIT clause for your solution.
> Since we can't just ORDER BY and LIMIT the cases, we can JOIN the table to an inner query version of itself that has only the MAX date selected.
>
> From the outer query we can SELECT the columns we want to return (i.e. first name, surname)

In [20]:
# query
q ="""
SELECT m.firstname AS first_name,
       m.surname AS last_name
FROM Members m
JOIN (SELECT MAX(mm.joindate) AS max_date
	FROM Members mm) t
ON t.max_date = m.joindate
"""
# DataFrame of return
df = pd.read_sql_query(q, con)
df

,first_name,last_name
0,Darren,Smith


### **Q7:** How can you produce a list of all members who have used a tennis court?
### Include in your output the name of the court, and the name of the member
### formatted as a single column. Ensure no duplicate data, and order by
### the member name.
> This one was a little tricky because the question syntax was initially confusing.
>
> Judgment-aside for the significant CTRL+V use, the query below uses the CASE WHEN clause to evaluate whether the member used Tennis Court 1, Tennis Court 2, or both - then returns the appropriate Facility Name string to the table.
>
> Since the questions asks after members specifically, GUESTS are weeded out in the outermost WHERE clause.

In [21]:
# query
q ="""
/* replace the following to go from SQL to SQLite3: 
ELSE CONCAT(m.firstname, ' ', m.surname)
ELSE m.firstname || ' ' || m.surname
*/

SELECT DISTINCT m.firstname || ' ' || m.surname AS member_name,
       CASE WHEN m.firstname IN (
                              SELECT tm.firstname
                                  FROM Bookings tb
                                  JOIN Members tm
                                    ON tb.memid = tm.memid 
                                  JOIN Facilities tf
                                    ON tb.facid = tf.facid
                                  WHERE tf.name = 'Tennis Court 1'
                                ) 
             AND m.surname IN (
                              SELECT tm.surname
                                  FROM Bookings tb
                                  JOIN Members tm
                                    ON tb.memid = tm.memid 
                                  JOIN Facilities tf
                                    ON tb.facid = tf.facid
                                  WHERE tf.name = 'Tennis Court 1'
                                )
             AND m.firstname IN (
                              SELECT tm.firstname
                                  FROM Bookings tb
                                  JOIN Members tm
                                    ON tb.memid = tm.memid 
                                  JOIN Facilities tf
                                    ON tb.facid = tf.facid
                                  WHERE tf.name = 'Tennis Court 2'
                                ) 
             AND m.surname IN (
                              SELECT tm.surname
                                  FROM Bookings tb
                                  JOIN Members tm
                                    ON tb.memid = tm.memid 
                                  JOIN Facilities tf
                                    ON tb.facid = tf.facid
                                  WHERE tf.name = 'Tennis Court 2'
                                ) 
            THEN 'Tennis Court 1, 2'
            WHEN m.firstname IN (
                              SELECT tm.firstname
                                  FROM Bookings tb
                                  JOIN Members tm
                                    ON tb.memid = tm.memid 
                                  JOIN Facilities tf
                                    ON tb.facid = tf.facid
                                  WHERE tf.name = 'Tennis Court 1'
                                ) 
            AND m.surname IN (
                              SELECT tm.surname
                                  FROM Bookings tb
                                  JOIN Members tm
                                    ON tb.memid = tm.memid 
                                  JOIN Facilities tf
                                    ON tb.facid = tf.facid
                                  WHERE tf.name = 'Tennis Court 1'
                                ) 
            THEN 'Tennis Court 1'
            WHEN m.firstname IN (
                              SELECT tm.firstname
                                  FROM Bookings tb
                                  JOIN Members tm
                                    ON tb.memid = tm.memid 
                                  JOIN Facilities tf
                                    ON tb.facid = tf.facid
                                  WHERE tf.name = 'Tennis Court 2'
                                ) 
            AND m.surname IN (
                              SELECT  tm.surname
                                  FROM Bookings tb
                                  JOIN Members tm
                                    ON tb.memid = tm.memid 
                                  JOIN Facilities tf
                                    ON tb.facid = tf.facid
                                  WHERE tf.name = 'Tennis Court 2'
                                ) 
            THEN 'Tennis Court 2'
        END AS booked_court
FROM Members m
JOIN Bookings b
ON m.memid = b.memid
JOIN Facilities f
ON f.facid = b.facid
WHERE f.name LIKE 'Tennis Court%'
AND ((m.firstname != 'GUEST') OR (m.surname != 'GUEST'))
ORDER BY member_name
"""
# DataFrame of return
df = pd.read_sql_query(q, con)
df

,member_name,booked_court
0,Anne Baker,"Tennis Court 1, 2"
1,Burton Tracy,"Tennis Court 1, 2"
2,Charles Owen,"Tennis Court 1, 2"
3,Darren Smith,Tennis Court 2
4,David Farrell,"Tennis Court 1, 2"
5,David Jones,"Tennis Court 1, 2"
6,David Pinker,Tennis Court 1
7,Douglas Jones,Tennis Court 1
8,Erica Crumpet,Tennis Court 1
9,Florence Bader,"Tennis Court 1, 2"


### **Q8:** How can you produce a list of bookings on the day of 2012-09-14 which
### will cost the member (or guest) more than $30? Remember that guests have
### different costs to members (the listed costs are per half-hour 'slot'), and
### the guest user's ID is always 0. Include in your output the name of the
### facility, the name of the member formatted as a single column, and the cost.
### Order by descending cost, and do not use any subqueries.
> To avoid using subqueries, we turn instead to the CASE WHEN clause again to evaluate whether the booking was done by a member or a guest, mutliply the cost by the number of slots requested, and return that value as a 3rd column `booking_cost` that we can later ORDER BY.
>
> In order to return *only* those bookings for 09-14-2012, I used a WHERE clause and conditional calls to get the day isolated.
>
> To restrict the booking orders returns to *only* those greater than 30, I used another WHERE clause combined with AND clauses to do the same logic evaluation used to evaluate cost earlier.
>
> *Because GUEST GUEST is distracting, we also used a CASE WHEN to return just the firstname GUEST when it was found in the firstname return.*

In [22]:
# query
q ="""
/* replace the following to go from SQL to SQLite3: 
ELSE CONCAT(m.firstname, ' ', m.surname)
ELSE m.firstname || ' ' || m.surname
*/

SELECT f.name AS facility_name,
           CASE WHEN m.firstname = 'GUEST' THEN m.firstname 
                ELSE m.firstname || ' ' || m.surname END AS member_name, -- here
           CASE WHEN ((b.memid > 0) AND ((b.slots * f.membercost) > 30)) THEN (b.slots * f.membercost)
                WHEN ((b.memid = 0) AND ((b.slots * f.guestcost) > 30)) THEN (b.slots * f.guestcost)
                ELSE 0 END AS booking_cost
        FROM Bookings b
        JOIN Members m
        ON m.memid = b.memid
        JOIN Facilities f
        ON f.facid = b.facid
    WHERE b.starttime >= '2012-09-14'
    AND b.starttime < '2012-09-15'
    AND (
        ((b.memid > 0) AND ((b.slots * f.membercost) > 30))
        OR 
        ((b.memid = 0) AND ((b.slots * f.guestcost) > 30))
        )
    ORDER BY booking_cost DESC
"""
# DataFrame of return
df = pd.read_sql_query(q, con)
df

,facility_name,member_name,booking_cost
0,Massage Room 2,GUEST,320.0
1,Massage Room 1,GUEST,160.0
2,Massage Room 1,GUEST,160.0
3,Massage Room 1,GUEST,160.0
4,Tennis Court 2,GUEST,150.0
5,Tennis Court 1,GUEST,75.0
6,Tennis Court 1,GUEST,75.0
7,Tennis Court 2,GUEST,75.0
8,Squash Court,GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


## **Q9:** This time, produce the same result as in Q8, but using a subquery.
> We use the subquery to establish the cost as a column we can reference in the outer query. 
> 
> Since we are selecting FROM the subquery, the calculation has already been run and the column established. 
> 
> This way, we can use it with the WHERE clause in the outer query to restrict the booking returns by price.

In [23]:
# query
q ="""
/* replace the following to go from SQL to SQLite3: 
ELSE CONCAT(dsub.firstname, ' ', dsub.surname) 
ELSE dsub.firstname || ' ' || dsub.surname
*/

SELECT dsub.name AS facility_name,
	   CASE WHEN dsub.firstname = 'GUEST' THEN dsub.firstname 
			ELSE dsub.firstname || ' ' || dsub.surname END AS member_name, -- here 
       dsub.cost AS booking_cost
FROM
  (
   SELECT b.bookid, b.facid, b.memid,
          m.surname, m.firstname,
          f.name, f.membercost, f.guestcost,
          CASE WHEN b.memid = 0 THEN b.slots * f.guestcost
            ELSE b.slots * f.membercost END AS cost
    FROM Bookings b
    JOIN Members m
    ON m.memid = b.memid
    JOIN Facilities f
    ON f.facid = b.facid
    WHERE b.starttime >= '2012-09-14'
    AND b.starttime < '2012-09-15'
  ) dsub
WHERE dsub.cost > 30
ORDER BY dsub.cost DESC
"""
# DataFrame of return
df = pd.read_sql_query(q, con)
df

,facility_name,member_name,booking_cost
0,Massage Room 2,GUEST,320.0
1,Massage Room 1,GUEST,160.0
2,Massage Room 1,GUEST,160.0
3,Massage Room 1,GUEST,160.0
4,Tennis Court 2,GUEST,150.0
5,Tennis Court 1,GUEST,75.0
6,Tennis Court 1,GUEST,75.0
7,Tennis Court 2,GUEST,75.0
8,Squash Court,GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


## **Q10:** Produce a list of facilities with a total revenue less than 1000.
## The output of facility name and total revenue, sorted by revenue. Remember
## that there's a different cost for guests and members!
> Like in question 09, we again use the subquery to establish values for the outer query. Since we are dealing with *total revenue*, we needed to calculate revenue and subtract costs for each booking, 
>
> For the revenue portion of the analysis, we used the CASE WHEN clause in the subquery to evaluate whether the member cost or guest cost is applied, then returned the value to the *rev* field. The revenue can then be summed in the outer query.
>
> The revenues need to be grouped into facility with GROUP BY, and those less than 1000 can be returned by applying the HAVING clause to the total_revenue. 
>
> To calculate costs, we assume that `initialoutlay` is an initial cost to setup the facility. Monthly maintenance needed to be multiplied by the number of months bookings occurred, so a substring of the booking start times is used to provide the year and month as separate fields. Then, a COUNT can be used in the outer query to return the total maintenance cost.
>
> Once the revenue and costs are aggregated in the outer query, the total_revenue column can be calculated.


In [24]:
# query
q ="""
/* 
replaced the following to go from SQL to SQLite3:
DISTINCT CONCAT(st.year, ' ', st.month)
DISTINCT st.year || ' ' || st.month

SUBSTRING(b.starttime, 6, 2) AS month
SUBSTR(b.starttime, 6, 2) AS month

SUBSTRING(b.starttime, 1, 4) AS year
SUBSTR(b.starttime, 1, 4) AS year
*/
SELECT st.fname AS facility_name,
       (SUM(st.rev) - of.initialoutlay - (of.monthlymaintenance * COUNT(DISTINCT st.year || ' ' || st.month))) AS total_revenue -- here  
FROM(
	SELECT f.name AS fname,
			b.bookid,
	       SUBSTR(b.starttime, 6, 2) AS month, -- here
       	   SUBSTR(b.starttime, 1, 4) AS year, -- here
	CASE WHEN b.memid = 0 THEN b.slots * f.guestcost
	ELSE b.slots * f.membercost END AS rev
	FROM Bookings b
	JOIN Facilities f
	ON b.facid = f.facid
	) st
JOIN Facilities of
ON st.fname = of.name
GROUP BY facility_name
HAVING total_revenue < 1000
ORDER BY total_revenue
"""
# DataFrame of return
df = pd.read_sql_query(q, con)
df

,facility_name,total_revenue
0,Badminton Court,-2243.5
1,Snooker Table,-255.0
2,Pool Table,-175.0
3,Table Tennis,-170.0


*Voila!* So we have it!

## Suggestions for Improvement
A few things come to mind when working through this project:
- More time could be spent working with datetime to get better value when needing to query by time periods
- Query optimization was neglected when trying to solve these problems
- A more robust understanding of the order of execution regarding clauses might help better design queries (And avoid unnecessary subqueries, joins, etc)
- More review on join, subquery, and *how* to use them in solving multi-step problems can help (Looking at you, Answer 7)